In [ ]:
%matplotlib inline 
#import plotting functions
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import csv
import warnings

In [ ]:
#load the metadata
metadata=pd.read_csv("$File location of metadata.csv", header=0)
metadata.tail(5)

In [ ]:
otu_rdp_assigned=pd.read_csv("$File location of taxa data.csv", header=0,index_col=0)
otu_rdp_assigned.transpose().head(5)

In [ ]:
#extract taxonomy information from the first column
otu_taxon=otu_rdp_assigned.iloc[:,0]
otu_taxon

In [ ]:
otu_rdp_assigned.shape

In [ ]:
#i skipped a lot of the code here because I did the preporcessing in R 
#get the sums since it is rarefyied it should be the same 
tot = otu_rdp_assigned.iloc[:,0:].sum(axis = 0)
tot

In [ ]:
#calc the rel abund
#otu_genus_frac=otu_rdp_assigned.iloc[:,1:].divide(tot,axis=1)
#otu_genus_frac.head(5)

otu_genus_frac=otu_rdp_assigned.divide(tot,axis=1)
otu_genus_frac.head()

In [ ]:
#first step is to reformat the data both the taxa and metadata
#otu_genus_frac.transpose().head(5)
otu_rdp_assigned.transpose()

In [ ]:
#Define the variables in your metadata
otu_model=otu_genus_frac.transpose()
otu_model=otu_model.reset_index()
otu_model=otu_model.rename(columns={'index':'$Sample_ID'})
dat=otu_model.merge(metadata[['$Sample_ID','+ any other variables in your metadata']],on = '$Sample_ID', how = 'inner')
dat #locational variables must be in some form of binary (ex. 0 and 1, -1 and 1, etc)

In [ ]:
###Packages required for RidgeClassiferCV and crossvalidation###
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeClassifierCV

#Cross validation 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from numpy import mean
from numpy import std
from numpy import absolute

In [ ]:
#Main portion to train and test the model
i = 0 # used for alternate method of cross validation

train_accu = []
test_accu = []
train_sum = 0
test_sum = 0 

while i <1: #ALT cross validation method, change this number to how ever many times you want to retrain the model 
    warnings.filterwarnings(action='ignore') #We do this to keep the ouput clean
    X_train, X_test, y_train, y_test = train_test_split(dat.drop(['$List of all unwanted varaible in dat file'], axis=1), dat['$Regional Variable of Interest'], test_size=0.2)
    ridge_model = RidgeClassifierCV(alphas=[0.01,0.05,0.1,0.5,1,2,5,10], normalize=True) #tests the various alphas and picks the best one.
    ridge_model.fit(X_train, y_train)
    train = ridge_model.score(X_train,y_train)
    test = ridge_model.score(X_test, y_test)
    
    train_accu.append(train)
    test_accu.append(test)
    
    i += 1

train_mean = str(np.mean(train_accu))
train_std = str(np.std(train_accu))
    
test_mean = str(np.mean(test_accu))
test_std = str(np.std(test_accu))

print("Number of iterations: " + str(i))

print("Mean Training Accuracy: " + train_mean + " ± " + train_std) #ALT cross validation method
               
print("Mean Test Accuracy: " + test_mean + " ± " + test_std) #ALT cross vlaidation method

# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(ridge_model, X_train, y_train, scoring='accuracy', cv=10, n_jobs=-1)
# force scores to be positive
scores = absolute(scores)

print('Cross Valdiation Accuracy is: %.3f (%.3f)' % (mean(scores), std(scores)))